In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
with open('penalties_train.json', 'r') as f:
    raw_data = f.readlines()
    
raw_data = map(lambda x: x.rstrip(), raw_data)
raw_data_str = '[' + ','.join(raw_data) + ']'
labeled_train_data = pd.read_json(raw_data_str)

In [3]:
with open('unmarked.json', 'r') as f:
    raw_data = f.readlines()
    
raw_data = map(lambda x: x.rstrip(), raw_data)
raw_data_str = '[' + ','.join(raw_data) + ']'
unlabeled_train_data = pd.read_json(raw_data_str)

In [4]:
with open('penalties_test.json', 'r') as f:
    raw_data = f.readlines()
    
raw_data = map(lambda x: x.rstrip(), raw_data)
raw_data_str = '[' + ','.join(raw_data) + ']'
test_data = pd.read_json(raw_data_str)

In [5]:
print len(unlabeled_train_data)
print len(labeled_train_data)
print len(test_data)

11753
10014
2400


In [6]:
unlabeled_train_data.head()

,id,sample,token
0,14,BETWEEN RUSSIA AND EUROPE\n CHANGES\n CHAN...,NaN
1,17,BETWEEN RUSSIA AND AREA 3\n CHANGES\n CHAN...,NaN
2,21,FOR NEX FARES\n CANCELLATIONS\n BEFORE DE...,NaN
3,33,BETWEEN MSQ AND MOW FOR ROUND TRIP FARES\n CA...,NaN
4,35,UNLESS OTHERWISE SPECIFIED\n FOR TRAVEL ON/BE...,NaN


In [39]:
unlabeled_sample = unlabeled_train_data['sample'].tolist()
labeled_sample = labeled_train_data['sample'].tolist()
test_sample = test_data['sample'].tolist()

##Токенизатор

In [1]:
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
r = RegexpTokenizer('\w+')
stop_words_ = stopwords.words('english') + stopwords.words('russian')

def processing_text(text):
    raw_sentences = sent_tokenize(text.strip())
    sentences = []
    for raw in raw_sentences:
        if len(raw) >0:
            no_stop = [i for i in r.tokenize(raw.lower()) if i not in stop_words_] #uncommented after Word2Vec model been taught
            sentences += no_stop #editted from .append to += after Word2Vec model done
    return sentences

In [2]:
processing_text('I was, ok but: this is! Not ok, man')

['ok', 'ok', 'man']

In [3]:
text = u'Привет, как дела?'

In [4]:
sent_tokenize(text.strip())

[u'\u041f\u0440\u0438\u0432\u0435\u0442, \u043a\u0430\u043a \u0434\u0435\u043b\u0430?']

In [5]:
raw = 'I was, ok but: this not!'
[i for i in r.tokenize(raw.lower()) if i not in stop_words_]

['ok']

##Токенизатор Kaggle (проблемы с русским)

In [7]:
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    review_text = re.sub("[^a-zA-Z]"," ", review)
    words = review_text.lower().split()
    return(words)

import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences( review, tokenizer ):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append( review_to_wordlist( raw_sentence ))
    return sentences

In [8]:
review_to_sentences('I was, ok but: this not! Get off, man', tokenizer)

[['i', 'was', 'ok', 'but', 'this', 'not'], ['get', 'off', 'man']]

In [9]:
review = u'Привет, как дела?'

In [10]:
tokenizer.tokenize(review.strip())

[u'\u041f\u0440\u0438\u0432\u0435\u0442, \u043a\u0430\u043a \u0434\u0435\u043b\u0430?']

In [11]:
review = u'Привет, как дела?'
re.sub("[^a-zA-Zа-яА-ЯёЁ]"," ", review).lower().split()

[]

##------------------------------------------------------------------------------------------------------------

In [9]:
sent = []
for i in unlabeled_sample:
    sent += processing_text(i)
    
for i in labeled_sample:
    sent += processing_text(i)

In [10]:
len(sent)

909923

In [21]:
from gensim.models import Word2Vec
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

In [22]:
%%time
model = Word2Vec(sent, workers=num_workers, size=num_features, \
                 min_count=min_word_count, \
                 window = context, sample=downsampling)

CPU times: user 6min 13s, sys: 35.7 s, total: 6min 48s
Wall time: 2min 42s


In [23]:
model.init_sims(replace=True)

In [24]:
model.doesnt_match("man woman child kitchen".split())

'man'

In [25]:
model.doesnt_match("france england germany berlin".split())

'france'

In [26]:
model.doesnt_match("paris berlin london austria".split())

ValueError: cannot select a word from an empty list

In [33]:
model.most_similar("usa")

[(u'itineraries', 0.593355655670166),
 (u'good', 0.4699913263320923),
 (u'ii', 0.4585806727409363),
 (u'website', 0.454235702753067),
 (u'counters', 0.4495977759361267),
 (u'rebooked', 0.4460126757621765),
 (u'commences', 0.444488525390625),
 (u'differ', 0.4353044927120209),
 (u'canada', 0.42385154962539673),
 (u'wholy', 0.41377708315849304)]

In [34]:
model.syn0.shape

(3761, 300)

In [36]:
model["flight"]

array([ -1.95222981e-02,  -9.25823525e-02,  -1.17884250e-02,
        -2.70562228e-02,   1.09305941e-02,   8.45116153e-02,
        -5.53298555e-02,  -6.25760257e-02,  -4.11186880e-03,
        -3.68745178e-02,   7.81741273e-03,   1.50685473e-05,
        -5.29269911e-02,   3.82249393e-02,  -5.52816354e-02,
        -2.57438459e-02,   6.45049801e-03,  -3.97516042e-02,
        -1.04024753e-01,   4.53763753e-02,   1.90500785e-02,
         1.53786251e-02,   2.12360336e-03,   3.10129859e-03,
        -9.06417668e-02,   3.66495214e-02,  -1.52401635e-02,
         3.59585788e-03,   4.97317612e-02,  -4.94841039e-02,
         1.43225752e-02,   2.04714239e-02,   1.39417220e-02,
         1.54720172e-02,  -1.06012728e-02,   7.34348670e-02,
         2.59272195e-02,   4.93043326e-02,  -6.37894645e-02,
        -5.52719608e-02,   4.31292728e-02,   1.36197321e-02,
        -5.89284021e-03,  -5.67552149e-02,   1.22092783e-01,
        -1.07678629e-01,   4.22914587e-02,   2.59863175e-02,
        -3.17546986e-02,

#Средние векторы

##Функции

In [49]:
def makeFeatureVec(words, model, n_features):
    f_v = np.zeros((n_features,), dtype="float32")
    n_w = 0.
    index2word_set = set(model.index2word)
    for word in words:
        if word in index2word_set:
            n_w += 1
            f_v = np.add(f_v, model[word])
    f_v = np.divide(f_v, n_w)
    return f_v

def text_to_vec(texts, model, n_features):
    i = 0.
    text_v = np.zeros((len(texts), n_features), dtype="float32")
    for text in texts:
        if i % 1000. == 0.:
            print(i)
        text_v[i] = makeFeatureVec(text, model, n_features)
        i += 1
    return text_v

In [54]:
clean_train_sample = []
for sample in labeled_sample:
    clean_train_sample.append(processing_text(sample))

In [55]:
len(clean_train_sample)

10014

##Функции Kaggle (то же самое)

In [65]:
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    index2word_set = set(model.index2word)
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0.
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       counter = counter + 1.
    return reviewFeatureVecs

In [57]:
trainDataVecs = getAvgFeatureVecs( clean_train_sample, model, num_features )

Review 0 of 10014
Review 1000 of 10014
Review 2000 of 10014
Review 3000 of 10014
Review 4000 of 10014
Review 5000 of 10014
Review 6000 of 10014
Review 7000 of 10014
Review 8000 of 10014
Review 9000 of 10014
Review 10000 of 10014


In [58]:
clean_test_sample = []
for sample in test_sample:
    clean_test_sample.append(processing_text(sample))

In [60]:
len(clean_test_sample)

2400

In [61]:
testDataVecs = getAvgFeatureVecs( clean_test_sample, model, num_features )

Review 0 of 2400
Review 1000 of 2400
Review 2000 of 2400


##Random Forest

In [66]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)

print "Fitting a random forest to labeled training data..."
forest = forest.fit(trainDataVecs, labeled_train_data['token'] )

result_rf = forest.predict(testDataVecs)

Fitting a random forest to labeled training data...


In [69]:
import json
test_data['token'] = result_rf
for i in range(len(test_data)):
    result = {}
    result['id'] = str(test_data['id'][i])
    result['sample'] = test_data['sample'][i]
    result['token'] = test_data['token'][i]
    if i < 2399:
        with open('w2v_avg_rf.json', 'a') as f:
            json.dump(result, f)
            f.write('\n')
    else:
        with open('w2v_avg_rf.json', 'a') as f:
            json.dump(result, f)

In [ ]:
0.6125

## Log Reg

In [70]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l1')
lr = lr.fit(trainDataVecs, labeled_train_data['token'] )

result_lr = lr.predict(testDataVecs)

In [71]:
test_data['token'] = result_lr
for i in range(len(test_data)):
    result = {}
    result['id'] = str(test_data['id'][i])
    result['sample'] = test_data['sample'][i]
    result['token'] = test_data['token'][i]
    if i < 2399:
        with open('w2v_avg_lr.json', 'a') as f:
            json.dump(result, f)
            f.write('\n')
    else:
        with open('w2v_avg_lr.json', 'a') as f:
            json.dump(result, f)

# Centroids

In [73]:
%%time
from sklearn.cluster import KMeans
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

CPU times: user 1min 6s, sys: 1.58 s, total: 1min 7s
Wall time: 36.2 s


In [77]:
word_centroid_map = dict(zip(model.index2word, idx))

In [80]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [84]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (labeled_train_data["sample"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for sample in clean_train_sample:
    train_centroids[counter] = create_bag_of_centroids( sample, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test_data["sample"].size, num_clusters), \
    dtype="float32" )

counter = 0
for sample in clean_test_sample:
    test_centroids[counter] = create_bag_of_centroids( sample, \
        word_centroid_map )
    counter += 1

In [85]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids, labeled_train_data['token'])
result = forest.predict(test_centroids)
test_data['token'] = result
for i in range(len(test_data)):
    result = {}
    result['id'] = str(test_data['id'][i])
    result['sample'] = test_data['sample'][i]
    result['token'] = test_data['token'][i]
    if i < 2399:
        with open('w2v_centr_rf.json', 'a') as f:
            json.dump(result, f)
            f.write('\n')
    else:
        with open('w2v_centr_rf.json', 'a') as f:
            json.dump(result, f)

Fitting a random forest to labeled training data...


In [ ]:
0.607083333333